# Question Answering Data Preparation

In [22]:
import html
import json
import pickle
import random
import sys

import pandas

sys.path.append('..')
from aips import *

engine = get_engine()
outdoors_collection = engine.get_collection("outdoors")

NOTE: This notebook depends upon the Outdoors dataset. If you have any issues, please rerun the [Setting up the Outdoors Dataset](../ch13/1.setting-up-the-outdoors-dataset.ipynb) notebook.

## Listing 14.4

In [2]:
def get_questions():
    question_types = ["who", "what", "when", "where", "why", "how"]
    questions = []
    for type in question_types:
        request = {
            "query": type,
            "query_fields": ["title"],
            "return_fields": ["id", "url", "owner_user_id", "title", "accepted_answer_id"],
            "filters": [("-accepted_answer_id", "0")],
            "limit": 10000
        }
        docs = outdoors_collection.search(**request)["docs"]
        questions += [d for d in docs
                      if d["title"].lower().startswith(type)]  #Only titles starting with a question type
    return questions

## Listing 14.5

In [3]:
def get_context_dataframe(questions):
    contexts = {"id": [], "question":[], "context":[], "url": []}
    for question in questions:
        print(question)
        request = {"query": "*",
                   "query_fields": ["body"],
                   "filters": [("id", str(question["accepted_answer_id"]))],
                   "limit": 1,
                   "order_by": [("score", "desc")]}
        docs = outdoors_collection.search(**request)["docs"]
        contexts["id"].append(question["id"])
        contexts["url"].append(question["url"])
        contexts["question"].append(question["title"]),
        contexts["context"].append(docs[0]["body"])
    return pandas.DataFrame(contexts)

In [4]:
questions = get_questions()
contexts = get_context_dataframe(questions)
contexts[0:10]

{'id': '4410', 'accepted_answer_id': 4421, 'owner_user_id': 2440, 'title': 'Who places the anchors that rock climbers use?', 'url': 'https://outdoors.stackexchange.com/questions/4410'}
{'id': '5347', 'accepted_answer_id': 5348, 'owner_user_id': 2989, 'title': 'Who places the bolts on rock climbing routes, and how?', 'url': 'https://outdoors.stackexchange.com/questions/5347'}
{'id': '20662', 'accepted_answer_id': 20668, 'owner_user_id': 381, 'title': 'Who gets the bill if you activate a PLB to help someone else?', 'url': 'https://outdoors.stackexchange.com/questions/20662'}
{'id': '7623', 'accepted_answer_id': 7649, 'owner_user_id': 2301, 'title': 'What knot is this one? What are its purposes?', 'url': 'https://outdoors.stackexchange.com/questions/7623'}
{'id': '11587', 'accepted_answer_id': 11595, 'owner_user_id': 2573, 'title': 'What sort of crane, and what sort of snake?', 'url': 'https://outdoors.stackexchange.com/questions/11587'}
{'id': '1660', 'accepted_answer_id': 1664, 'owner_u

,id,question,context,url
0,4410,Who places the anchors that rock climbers use?,There are two distinct styles of free rock cli...,https://outdoors.stackexchange.com/questions/4410
1,5347,"Who places the bolts on rock climbing routes, ...",What you're talking about is Sport climbing. G...,https://outdoors.stackexchange.com/questions/5347
2,20662,Who gets the bill if you activate a PLB to hel...,"Almost always the victim gets the bill, but as...",https://outdoors.stackexchange.com/questions/2...
3,7623,What knot is this one? What are its purposes?,Slip knot It's undoubtably a slip knot that's ...,https://outdoors.stackexchange.com/questions/7623
4,11587,"What sort of crane, and what sort of snake?","To answer the snake part of it, looking at som...",https://outdoors.stackexchange.com/questions/1...
5,1660,What is Geocaching?,"In short, it's a high-tech treasure hunt. geoc...",https://outdoors.stackexchange.com/questions/1660
6,10173,What is a longbow?,The problem with the longbow discussion is tha...,https://outdoors.stackexchange.com/questions/1...
7,15947,"What are ""burnable gasses""?",Wood is not a simple chemical. It releases a n...,https://outdoors.stackexchange.com/questions/1...
8,913,What is a buff?,To be honest I was dubious about getting somet...,https://outdoors.stackexchange.com/questions/913
9,18369,"What are ""minimalist shoes""?",There was actually a project done on the defin...,https://outdoors.stackexchange.com/questions/1...


In [5]:
contexts.to_csv("data/outdoors/question-answer-seed-contexts.csv", index=False)

## Listing 14.6

In [6]:
import torch

def get_processor_device():
    return 0 if torch.cuda.is_available() else -1

In [7]:
from transformers import pipeline
import tqdm

model_name = "deepset/roberta-base-squad2"
pipeline_type = "question-answering"

device = get_processor_device()

def answer_questions(contexts, k=10):
    nlp = pipeline(pipeline_type, model=model_name,
                   tokenizer=model_name, device=device)
    guesses = []
    for _, row in tqdm.tqdm(contexts[0:k].iterrows(), total=k):
        result = nlp({"question": row["question"], "context": row["context"]})
        guesses.append(result)
    return guesses

In [8]:
guesses = answer_questions(contexts, k=len(contexts))
contexts["answers"] = guesses
guesses[0:10]

100%|██████████| 1663/1663 [07:43<00:00,  3.58it/s]


[{'score': 0.27892693877220154,
  'start': 474,
  'end': 516,
  'answer': 'a local enthusiast or group of enthusiasts'},
 {'score': 0.20084883272647858,
  'start': 81,
  'end': 117,
  'answer': 'the person who is creating the climb'},
 {'score': 0.01863168738782406,
  'start': 14,
  'end': 24,
  'answer': 'the victim'},
 {'score': 0.22231701016426086, 'start': 29, 'end': 38, 'answer': 'slip knot'},
 {'score': 0.000551236851606518,
  'start': 1255,
  'end': 1262,
  'answer': 'aquatic'},
 {'score': 0.3749975562095642,
  'start': 15,
  'end': 40,
  'answer': 'a high-tech treasure hunt'},
 {'score': 0.24700771272182465,
  'start': 227,
  'end': 265,
  'answer': 'the traditional longbow made from wood'},
 {'score': 8.70941803441383e-05,
  'start': 695,
  'end': 706,
  'answer': 'water vapor'},
 {'score': 0.5637540221214294,
  'start': 192,
  'end': 232,
  'answer': 'a tube of lightweight, stretchy material'},
 {'score': 0.48040667176246643,
  'start': 408,
  'end': 473,
  'answer': 'shoes i

In [10]:
contexts.to_csv("data/outdoors/question-answer-squad2-guesses.csv", index=False)

## Listing 14.7

### ***Manually labeling data**
*The above csv file (../data/question-answer-squad2-guesses.csv) is used as a raw first pass at attempting to answer the questions.  This is then used with human-in-the-loop manual correction and labelling of the data.  There is no python code that can do this for you.  The data MUST be labelled by an intelligent person with an understanding of the domain.  All further listings will use the 'golden set' - the manually corrected answer file, and not the guesses that were generated above.*

In [24]:
from datasets import Dataset, DatasetDict
random.seed(0)

def get_training_data(filename):
    golden_answers = pandas.read_csv(filename)
    golden_answers["class"] = golden_answers["class"].fillna(-2).astype(int)
    validated = golden_answers[golden_answers["class"] > -1]    
    table = {"id": [], "url": [], "title": [], "question": [], "context": [], "answers": []}
    
    for _, row in validated.iterrows():
        answers = row["gold"].split("|")
        starts = []
        missing = False
        for answer in answers:
            result = row["context"].find(answer)
            starts.append(result)
            if(result < 0):
                missing = True
        if not missing:
            table["id"].append(row["id"])
            table["url"].append(row["url"])
            table["title"].append(row["question"])
            table["question"].append(row["question"])
            table["context"].append(row["context"])
            table["answers"].append({"text": answers, "answer_start": starts})
            
    df = pandas.DataFrame(table).sample(frac=1)
    train_split = int(len(df) * 0.75)
    eval_split = int((len(df) - train_split) / 1.25) + train_split - 1
    train_dataset = Dataset.from_pandas(df[:train_split])
    test_dataset = Dataset.from_pandas(df[train_split:eval_split])
    validation_dataset = Dataset.from_pandas(df[eval_split:])
    return DatasetDict({"train": train_dataset, "test": test_dataset, "validation": validation_dataset})

#This golden answers file was labeled by me (Max Irwin).
#It took about 2-3 hours to label 200 question/answer rows
#Doing so will give you a deeper appreciation for the difficulty of the NLP task.
#I *highly* encourage you to label even more documents, and re-run the fine-tuning tasks coming up.
data = get_training_data("../data/outdoors/outdoors_golden_answers_20210130.csv")
data.save_to_disk("../data/outdoors/question-answering-training-set")
data

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'question', 'context', 'answers', '__index_level_0__'],
        num_rows: 125
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'question', 'context', 'answers', '__index_level_0__'],
        num_rows: 32
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'question', 'context', 'answers', '__index_level_0__'],
        num_rows: 10
    })
})

Up next: [Question Answering LLM Fine-tuning](3.question-answering-GPU-fine-tuning.ipynb)